In [1]:
from ipyopenlayers import Map, RasterTileLayer, GeoJSON
from ipywidgets import Output, HBox, VBox, HTML, Text
import bqplot.pyplot as plt
from bqplot import DateScale, LinearScale,OrdinalScale, Bars, Axis, Figure,CATEGORY10
from geopy.geocoders import Nominatim
import asyncio
from ipyflex import FlexLayout 
import json 
import os
import configparser
import random
import requests
import Functions as fct
import ipywidgets as widgets 
try:
    config = configparser.ConfigParser() 
    config.read('.config')
    token = config['DEFAULT']['token']
except:
    token = ""

In [2]:
if not os.path.exists('countries.geojson'):
    url = 'https://github.com/datasets/geo-countries/blob/master/data/countries.geojson'
    r = requests.get(url)
    with open('countries_copy', 'w') as f:
        f.write(r.content.decode("utf-8"))
 
with open('countries.geojson', 'r') as f:
    data = json.load(f)
def random_color(feature):
    return {
        'color': 'black',
        'fillColor': random.choice(['red', 'yellow', 'green', 'orange']),
    }
geo_json = GeoJSON(
    data=data,
    style={
        'opacity': 1, 'dashArray': '9', 'fillOpacity': 0.1, 'weight': 1
    },
    hover_style={
        'color': 'white', 'dashArray': '0', 'fillOpacity': 0.5
    },
    style_callback=random_color
)
m = Map(center=(50.6252978589571, 0.34580993652344), zoom=2)
layer=RasterTileLayer()
m.add_layer(layer)

geo_json_layer = GeoJSON(data=geo_json.data)
m.add_layer(geo_json_layer)


In [3]:
box = VBox()
axes_options_1 = {
    "x": {"label": "Time","label_style": {"font-size": "3px"}},
    "y": {"label": "Power Consumption (MWh)", "label_offset": "6ex","label_font_size": "3px"},
}
axes_options_2 = {
    "x": {"label": "Time","label_offset": "6ex"},
    "y": {"label": "Power Value", "label_offset": "6ex"},
}
axes_options_3 = {
    "x": {"label": "Source Energie","label_offset": "6ex"},
    "y": {"label": "Value (MWh)", "label_offset": "6ex"},
}
axes_options_4 = {
    "x": {"label": "Country","label_offset": "6ex"},
    "y": {"label": "Value (MWh)", "label_offset": "6ex"},
}


In [8]:
# Load API key from configuration
config = configparser.ConfigParser()
config.read('.env.ini')
api_key = config['DEFAULT']['api_key']

# Initialize the plots with correct configurations
# Hourly Power Consumption Chart
x_ord1 = DateScale()
y_sc1 = LinearScale()

bar = Bars(scales={'x': x_ord1, 'y': y_sc1}, colors=['blue'], padding=0.2)

ax_x = Axis(scale=x_ord1, **axes_options_1["x"])
ax_y = Axis(scale=y_sc1, orientation='vertical', tick_format='0.1e', **axes_options_1["y"])

Hourly_Power_Consumption_fig = Figure(marks=[bar], axes=[ax_x, ax_y], 
             animation_duration=1000, legend_style=dict(fill="var(--jp-layout-color0)"),
             legend_location="bottom-left")
Hourly_Power_Consumption_fig.axes[0].tick_style = {'font-size': '10px'}  
Hourly_Power_Consumption_fig.axes[1].tick_style = {'font-size': '10px'} 
Hourly_Power_Consumption_fig.layout.width = '100%'  
axes = plt.axes(options=axes_options_1)

# Import Export Chart
x_scale = DateScale()
y_scale = LinearScale()

Exp_bar = Bars(scales={"x": x_scale, "y": y_scale}, padding=0.2,  colors=['blue'])
Imp_bar = Bars(scales={"x": x_scale, "y": y_scale}, padding=0.2, colors=['red'])

ax_x = Axis(scale=x_scale,**axes_options_2["x"])
ax_y = Axis(scale=y_scale, orientation="vertical", tick_format="0.2f",**axes_options_2["y"])

Exp_fig = Figure(axes=[ax_x, ax_y], animation_duration=1000)
Imp_fig=Figure(axes=[ax_x, ax_y], animation_duration=1000)

Exp_fig.marks = [Exp_bar]
Imp_fig.marks = [Imp_bar]

fig_exp_imp = widgets.HBox([Exp_fig, Imp_fig])

Exp_fig.axes[0].tick_style = {'font-size': '10px'}  
Exp_fig.axes[1].tick_style = {'font-size': '10px'}  
Imp_fig.axes[0].tick_style = {'font-size': '10px'}
Imp_fig.axes[1].tick_style = {'font-size': '10px'}
Exp_fig.layout.width = '100%'  
Imp_fig.layout.width = '100%'  

fig_exp_imp.layout.width = '100%'   

# Power by Source Chart
x_scale = OrdinalScale()
y_scale = LinearScale()

bar_S = Bars(scales={"x": x_scale, "y": y_scale}, padding=0.2, colors=CATEGORY10, color_mode="group")

ax_x = Axis(scale=x_scale, **axes_options_3["x"])
ax_y = Axis(scale=y_scale, orientation="vertical", tick_format="0.2f", **axes_options_3["y"])

bar_S.orientation = "horizontal"
ax_x.orientation = "vertical"
ax_y.orientation = "horizontal"

source_fig = Figure(axes=[ax_x, ax_y], animation_duration=1000, 
                    axes_options=axes_options_3)
source_fig.axes[0].tick_style = {'font-size': '6px', 'font-weight': 'bold'}  # Pour l'axe x
source_fig.axes[1].tick_style = {'font-size': '10px'}  # Pour l'axe x

source_fig.marks = [bar_S]

# Power Imported by Country Chart
x_ord3 = OrdinalScale()
y_sc3 = LinearScale()

bar_C = Bars(scales={"x": x_ord3, "y": y_sc3}, padding=0.1, colors=CATEGORY10, color_mode="group")
ax_x = Axis(scale=x_ord3,**axes_options_4["x"])
ax_y = Axis(scale=y_sc3, orientation="vertical", tick_format="0.2f",**axes_options_4["y"])

source_fig_c = Figure(axes=[ax_x, ax_y], animation_duration=1000, 
                    axes_options=axes_options_3)
source_fig_c.axes[0].tick_style = {'font-size': '10px'}  
source_fig_c.axes[1].tick_style = {'font-size': '10px'}  # Pour l'axe x
source_fig_c.marks = [bar_C]


# Function to handle click events
def handle_click(lon, lat, country, api_key):
    global country_variable
    country_variable = country
    print(f"Updated country variable to: {country_variable}")

    # Update titles with the clicked country
    Hourly_Power_Consumption_fig.title = f"Hourly Power Consumption {country}"
    fig_exp_imp.title = f"Hourly Power Consumption {country}"
    Exp_fig.title = f"Power Export of {country}"
    Imp_fig.title = f"Power Import of {country}"
    source_fig.title = f"Power by Source {country}"
    source_fig_c.title = f"Power Import by Country {country}"

    # Create asynchronous tasks for fetching data
    task = asyncio.create_task(fct.Hourly_Power_Consumption(country, bar, api_key))
    task1 = asyncio.create_task(fct.Hourly_Imp_Exp_Power_Consumption(country, Imp_bar, Exp_bar, api_key))
    task2 = asyncio.create_task(fct.Power_Consumption_Source(country, bar_S, api_key))
    task3 = asyncio.create_task(fct.Power_Imported_Country(country, bar_C, api_key))

def create_click_handler(api_key):
    def click_handler(lon, lat, country):
        print(f"API Key inside handler: {api_key}")
        handle_click(lon, lat, country, api_key)
    return click_handler
m.on_click(create_click_handler(api_key),api_key)

In [9]:
def handle_text_change(change):
    global token
    token = change['new']

def handle_button_click(button):
    global token
    token = text_widget.value
    text_widget.value = ''  

text_widget = widgets.Text(
    value='',
    placeholder='Enter your token here...',
    description='Token:',
    style={'description_width': 'initial'},
    layout={'width': '50%', 'margin': '10px auto'},
    continuous_update=False  
)

button = widgets.Button(
    description='Enter',
    button_style='success',
    layout={'margin': '0 auto'}
)

text_widget.observe(handle_text_change, names='value')
button.on_click(handle_button_click)

Token=HBox([text_widget, button])


In [10]:
info_text = HTML(
    value=(
        '<div style="padding: 20px; background-color: #2c3e50; border-radius: 10px; box-shadow: 0px 4px 10px rgba(0, 0, 0, 0.5);">'
        '<p style="color: #ecf0f1; font-style: italic; text-align: center; font-size: 16px; margin: 0;">'
        'You can get your token at this link: '
        '<a href="https://api-portal.electricitymaps.com/catalog/prd_8e3ood60rcv8f51h" target="_blank" style="color: #1abc9c; text-decoration: none; font-weight: bold;">'
        'https://api-portal.electricitymaps.com/catalog/prd_8e3ood60rcv8f51h</a></p>'
        '</div>'
    )
)
style = widgets.HTML("""<style>.js-plotly-plot {height: 100%;}</style> """)

Token_widget=VBox([info_text,Token])

In [11]:
all_widgets =  {'Info':Token_widget,'Map':m,'Hourly Power Consumption':Hourly_Power_Consumption_fig,'Export Import Power':fig_exp_imp, 'Power By Source': source_fig , 'Power Importer By Country': source_fig_c}
header = dict(
    title='Electricity Map Dashboard',
    style={
        'background': 'linear-gradient(to right, #0f2027, #203a43, #2c5364)',  
        'color': 'white',
        'fontStyle': 'italic',
        'padding': '10px',
        'fontSize': '24px',
        'textAlign': 'center'
    },
    buttons=['import', 'export']
)
w = FlexLayout(
    all_widgets, 
    style={'height': '1300px', 'borderTop': '5px'},
    template='temp.json',
    editable=True, 
    header=header
)
w

FlexLayout(children={'Info': VBox(children=(HTML(value='<div style="padding: 20px; background-color: #2c3e50; …